# Large-Scale Stochastic Variational GP Regression (CUDA) (w/ SVGP)

## Overview

In this notebook, we'll give an overview of how to use Deep Kernel Learning with SVGP stochastic variational regression to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In contrast to the SVDKL_Regression_GridInterp_CUDA notebook, we'll be using SVGP (https://arxiv.org/pdf/1411.2005.pdf) here to learn the inducing point locations. Our implementation of SVGP is modified to be efficient with the inference techniques used in GPyTorch.

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [4]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
        self.add_module('bn1', torch.nn.BatchNorm1d(1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 1000))
        self.add_module('bn2', torch.nn.BatchNorm1d(1000))
        self.add_module('relu2', torch.nn.ReLU())                       
        self.add_module('linear3', torch.nn.Linear(1000, 500))
        self.add_module('bn3', torch.nn.BatchNorm1d(500))
        self.add_module('relu3', torch.nn.ReLU())                  
        self.add_module('linear4', torch.nn.Linear(500, 50))       
        self.add_module('bn4', torch.nn.BatchNorm1d(50))
        self.add_module('relu4', torch.nn.ReLU())                  
        self.add_module('linear5', torch.nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()
# num_features is the number of final features extracted by the neural network, in this case 2.
num_features = 2

## Defining the GP Regression Layer

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [18]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import WhitenedVariationalStrategy

class GPRegressionLayer(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = WhitenedVariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPRegressionLayer, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Defining the DKL Model

With the feature extractor and GP regression layer defined, we can now define our full model. To do this, we simply create a module whose `forward()` method passes the data first through the feature extractor, and then through the GP regression layer.

The only other interesting feature of the model below is that we use a helper function, `scale_to_bounds`, to ensure that the features extracted by the neural network fit within the grid bounds used for SKI.

In [19]:
class DKLModel(gpytorch.Module):
    def __init__(self, inducing_points, feature_extractor, num_features):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GPRegressionLayer(inducing_points)
        self.num_features = num_features

    def forward(self, x):
        features = self.feature_extractor(x)
        res = self.gp_layer(features)
        return res
inducing_points = feature_extractor(train_x[:500, :])
model = DKLModel(inducing_points=inducing_points, feature_extractor=feature_extractor, num_features=num_features).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [20]:
model.train()
likelihood.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 6

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.gp_layer.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data=train_y.size(0), combine_terms=False)

# We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
# effective for VI.
with gpytorch.settings.max_cg_iterations(45):
    for i in range(num_epochs):
        scheduler.step()
        # Within each iteration, we will go over each minibatch of data
        for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            output = model(x_batch)
            # with combine_terms=False, we get the terms of the ELBO separated so we can print them individually if we'd like.
            # loss = -mll(output, y_batch) would also work.
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

            loss.backward()
            optimizer.step()

Epoch 1 [0/340] - Loss: 254.669 [-254.669, -0.000, 0.000]
Epoch 1 [1/340] - Loss: 527.052 [-527.051, 0.002, 0.000]
Epoch 1 [2/340] - Loss: 245.219 [-245.219, 0.000, 0.000]
Epoch 1 [3/340] - Loss: 301.289 [-301.289, 0.000, 0.000]
Epoch 1 [4/340] - Loss: 363.732 [-363.730, 0.001, 0.000]
Epoch 1 [5/340] - Loss: 339.470 [-339.468, 0.001, 0.000]
Epoch 1 [6/340] - Loss: 303.123 [-303.122, 0.001, 0.000]
Epoch 1 [7/340] - Loss: 250.182 [-250.181, 0.001, 0.000]
Epoch 1 [8/340] - Loss: 236.911 [-236.911, 0.001, 0.000]
Epoch 1 [9/340] - Loss: 229.205 [-229.204, 0.001, 0.000]
Epoch 1 [10/340] - Loss: 214.676 [-214.675, 0.001, 0.000]
Epoch 1 [11/340] - Loss: 233.456 [-233.455, 0.001, 0.000]
Epoch 1 [12/340] - Loss: 208.461 [-208.460, 0.001, 0.000]
Epoch 1 [13/340] - Loss: 224.239 [-224.238, 0.001, 0.000]
Epoch 1 [14/340] - Loss: 209.891 [-209.890, 0.002, 0.000]
Epoch 1 [15/340] - Loss: 248.116 [-248.114, 0.002, 0.000]
Epoch 1 [16/340] - Loss: 227.239 [-227.236, 0.002, 0.000]
Epoch 1 [17/340] - Loss

Epoch 1 [144/340] - Loss: 72.167 [-72.098, 0.069, 0.000]
Epoch 1 [145/340] - Loss: 76.684 [-76.614, 0.071, 0.000]
Epoch 1 [146/340] - Loss: 71.917 [-71.845, 0.072, 0.000]
Epoch 1 [147/340] - Loss: 72.265 [-72.193, 0.072, 0.000]
Epoch 1 [148/340] - Loss: 69.017 [-68.947, 0.071, 0.000]
Epoch 1 [149/340] - Loss: 92.182 [-92.113, 0.069, 0.000]
Epoch 1 [150/340] - Loss: 69.273 [-69.209, 0.065, 0.000]
Epoch 1 [151/340] - Loss: 67.052 [-66.990, 0.062, 0.000]
Epoch 1 [152/340] - Loss: 66.042 [-65.981, 0.060, 0.000]
Epoch 1 [153/340] - Loss: 75.603 [-75.543, 0.060, 0.000]
Epoch 1 [154/340] - Loss: 73.625 [-73.565, 0.060, 0.000]
Epoch 1 [155/340] - Loss: 68.331 [-68.271, 0.061, 0.000]
Epoch 1 [156/340] - Loss: 67.344 [-67.282, 0.062, 0.000]
Epoch 1 [157/340] - Loss: 69.817 [-69.754, 0.063, 0.000]
Epoch 1 [158/340] - Loss: 70.942 [-70.877, 0.064, 0.000]
Epoch 1 [159/340] - Loss: 70.125 [-70.060, 0.065, 0.000]
Epoch 1 [160/340] - Loss: 60.768 [-60.702, 0.067, 0.000]
Epoch 1 [161/340] - Loss: 66.91

Epoch 1 [288/340] - Loss: 52.343 [-52.194, 0.149, 0.000]
Epoch 1 [289/340] - Loss: 54.095 [-53.946, 0.149, 0.000]
Epoch 1 [290/340] - Loss: 48.858 [-48.709, 0.149, 0.000]
Epoch 1 [291/340] - Loss: 46.748 [-46.601, 0.147, 0.000]
Epoch 1 [292/340] - Loss: 47.912 [-47.767, 0.145, 0.000]
Epoch 1 [293/340] - Loss: 49.112 [-48.969, 0.143, 0.000]
Epoch 1 [294/340] - Loss: 46.974 [-46.834, 0.141, 0.000]
Epoch 1 [295/340] - Loss: 57.460 [-57.318, 0.142, 0.000]
Epoch 1 [296/340] - Loss: 52.153 [-52.012, 0.141, 0.000]
Epoch 1 [297/340] - Loss: 53.858 [-53.717, 0.141, 0.000]
Epoch 1 [298/340] - Loss: 47.157 [-47.015, 0.142, 0.000]
Epoch 1 [299/340] - Loss: 43.284 [-43.142, 0.142, 0.000]
Epoch 1 [300/340] - Loss: 51.673 [-51.531, 0.142, 0.000]
Epoch 1 [301/340] - Loss: 51.866 [-51.724, 0.141, 0.000]
Epoch 1 [302/340] - Loss: 48.650 [-48.508, 0.141, 0.000]
Epoch 1 [303/340] - Loss: 54.309 [-54.167, 0.141, 0.000]
Epoch 1 [304/340] - Loss: 45.414 [-45.274, 0.140, 0.000]
Epoch 1 [305/340] - Loss: 43.80

Epoch 2 [94/340] - Loss: 35.430 [-35.306, 0.124, 0.000]
Epoch 2 [95/340] - Loss: 31.525 [-31.401, 0.124, 0.000]
Epoch 2 [96/340] - Loss: 36.857 [-36.735, 0.123, 0.000]
Epoch 2 [97/340] - Loss: 38.996 [-38.875, 0.122, 0.000]
Epoch 2 [98/340] - Loss: 35.382 [-35.261, 0.121, 0.000]
Epoch 2 [99/340] - Loss: 37.845 [-37.725, 0.120, 0.000]
Epoch 2 [100/340] - Loss: 34.018 [-33.899, 0.119, 0.000]
Epoch 2 [101/340] - Loss: 35.397 [-35.279, 0.118, 0.000]
Epoch 2 [102/340] - Loss: 33.978 [-33.861, 0.117, 0.000]
Epoch 2 [103/340] - Loss: 35.799 [-35.682, 0.117, 0.000]
Epoch 2 [104/340] - Loss: 33.652 [-33.535, 0.117, 0.000]
Epoch 2 [105/340] - Loss: 38.715 [-38.598, 0.117, 0.000]
Epoch 2 [106/340] - Loss: 39.066 [-38.949, 0.117, 0.000]
Epoch 2 [107/340] - Loss: 35.962 [-35.845, 0.117, 0.000]
Epoch 2 [108/340] - Loss: 36.599 [-36.482, 0.117, 0.000]
Epoch 2 [109/340] - Loss: 30.084 [-29.967, 0.117, 0.000]
Epoch 2 [110/340] - Loss: 35.468 [-35.351, 0.117, 0.000]
Epoch 2 [111/340] - Loss: 35.803 [-35

Epoch 2 [238/340] - Loss: 31.038 [-30.912, 0.126, 0.000]
Epoch 2 [239/340] - Loss: 27.587 [-27.460, 0.127, 0.000]
Epoch 2 [240/340] - Loss: 36.179 [-36.051, 0.128, 0.000]
Epoch 2 [241/340] - Loss: 29.272 [-29.144, 0.128, 0.000]
Epoch 2 [242/340] - Loss: 28.708 [-28.579, 0.128, 0.000]
Epoch 2 [243/340] - Loss: 29.485 [-29.357, 0.128, 0.000]
Epoch 2 [244/340] - Loss: 27.342 [-27.214, 0.128, 0.000]
Epoch 2 [245/340] - Loss: 28.268 [-28.142, 0.126, 0.000]
Epoch 2 [246/340] - Loss: 27.416 [-27.290, 0.126, 0.000]
Epoch 2 [247/340] - Loss: 31.676 [-31.551, 0.125, 0.000]
Epoch 2 [248/340] - Loss: 29.596 [-29.471, 0.125, 0.000]
Epoch 2 [249/340] - Loss: 31.240 [-31.115, 0.125, 0.000]
Epoch 2 [250/340] - Loss: 28.444 [-28.318, 0.126, 0.000]
Epoch 2 [251/340] - Loss: 25.998 [-25.872, 0.126, 0.000]
Epoch 2 [252/340] - Loss: 28.639 [-28.513, 0.126, 0.000]
Epoch 2 [253/340] - Loss: 31.256 [-31.130, 0.126, 0.000]
Epoch 2 [254/340] - Loss: 30.495 [-30.369, 0.126, 0.000]
Epoch 2 [255/340] - Loss: 27.87

...

Epoch 5 [88/340] - Loss: 16.864 [-16.747, 0.117, 0.000]
Epoch 5 [89/340] - Loss: 19.649 [-19.532, 0.117, 0.000]
Epoch 5 [90/340] - Loss: 18.513 [-18.396, 0.117, 0.000]
Epoch 5 [91/340] - Loss: 18.002 [-17.885, 0.117, 0.000]
Epoch 5 [92/340] - Loss: 17.994 [-17.877, 0.117, 0.000]
Epoch 5 [93/340] - Loss: 17.250 [-17.133, 0.117, 0.000]
Epoch 5 [94/340] - Loss: 17.099 [-16.982, 0.117, 0.000]
Epoch 5 [95/340] - Loss: 21.212 [-21.095, 0.117, 0.000]
Epoch 5 [96/340] - Loss: 20.622 [-20.505, 0.117, 0.000]
Epoch 5 [97/340] - Loss: 18.194 [-18.077, 0.117, 0.000]
Epoch 5 [98/340] - Loss: 20.133 [-20.016, 0.117, 0.000]
Epoch 5 [99/340] - Loss: 19.082 [-18.965, 0.117, 0.000]
Epoch 5 [100/340] - Loss: 20.163 [-20.046, 0.117, 0.000]
Epoch 5 [101/340] - Loss: 20.308 [-20.191, 0.117, 0.000]
Epoch 5 [102/340] - Loss: 21.206 [-21.089, 0.117, 0.000]
Epoch 5 [103/340] - Loss: 19.858 [-19.741, 0.117, 0.000]
Epoch 5 [104/340] - Loss: 18.856 [-18.739, 0.117, 0.000]
Epoch 5 [105/340] - Loss: 22.710 [-22.593, 

Epoch 5 [232/340] - Loss: 17.409 [-17.291, 0.118, 0.000]
Epoch 5 [233/340] - Loss: 20.539 [-20.421, 0.118, 0.000]
Epoch 5 [234/340] - Loss: 17.979 [-17.861, 0.118, 0.000]
Epoch 5 [235/340] - Loss: 18.703 [-18.585, 0.118, 0.000]
Epoch 5 [236/340] - Loss: 16.684 [-16.566, 0.118, 0.000]
Epoch 5 [237/340] - Loss: 24.436 [-24.318, 0.118, 0.000]
Epoch 5 [238/340] - Loss: 19.456 [-19.338, 0.118, 0.000]
Epoch 5 [239/340] - Loss: 20.461 [-20.343, 0.118, 0.000]
Epoch 5 [240/340] - Loss: 18.967 [-18.849, 0.118, 0.000]
Epoch 5 [241/340] - Loss: 18.738 [-18.621, 0.117, 0.000]
Epoch 5 [242/340] - Loss: 17.173 [-17.055, 0.118, 0.000]
Epoch 5 [243/340] - Loss: 19.450 [-19.332, 0.118, 0.000]
Epoch 5 [244/340] - Loss: 19.956 [-19.838, 0.118, 0.000]
Epoch 5 [245/340] - Loss: 19.654 [-19.536, 0.118, 0.000]
Epoch 5 [246/340] - Loss: 17.665 [-17.547, 0.118, 0.000]
Epoch 5 [247/340] - Loss: 23.226 [-23.108, 0.118, 0.000]
Epoch 5 [248/340] - Loss: 18.604 [-18.486, 0.118, 0.000]
Epoch 5 [249/340] - Loss: 19.87

Epoch 6 [37/340] - Loss: 19.011 [-18.892, 0.118, 0.000]
Epoch 6 [38/340] - Loss: 17.451 [-17.333, 0.118, 0.000]
Epoch 6 [39/340] - Loss: 24.848 [-24.730, 0.118, 0.000]
Epoch 6 [40/340] - Loss: 18.352 [-18.234, 0.118, 0.000]
Epoch 6 [41/340] - Loss: 17.353 [-17.235, 0.118, 0.000]
Epoch 6 [42/340] - Loss: 17.335 [-17.217, 0.118, 0.000]
Epoch 6 [43/340] - Loss: 18.506 [-18.387, 0.118, 0.000]
Epoch 6 [44/340] - Loss: 19.395 [-19.277, 0.118, 0.000]
Epoch 6 [45/340] - Loss: 18.769 [-18.651, 0.118, 0.000]
Epoch 6 [46/340] - Loss: 19.972 [-19.854, 0.118, 0.000]
Epoch 6 [47/340] - Loss: 17.114 [-16.995, 0.119, 0.000]
Epoch 6 [48/340] - Loss: 17.560 [-17.442, 0.118, 0.000]
Epoch 6 [49/340] - Loss: 17.855 [-17.736, 0.118, 0.000]
Epoch 6 [50/340] - Loss: 17.581 [-17.463, 0.118, 0.000]
Epoch 6 [51/340] - Loss: 18.829 [-18.711, 0.118, 0.000]
Epoch 6 [52/340] - Loss: 19.825 [-19.707, 0.118, 0.000]
Epoch 6 [53/340] - Loss: 16.840 [-16.721, 0.118, 0.000]
Epoch 6 [54/340] - Loss: 17.547 [-17.428, 0.119,

Epoch 6 [182/340] - Loss: 19.166 [-19.047, 0.119, 0.000]
Epoch 6 [183/340] - Loss: 17.723 [-17.605, 0.119, 0.000]
Epoch 6 [184/340] - Loss: 19.033 [-18.914, 0.119, 0.000]
Epoch 6 [185/340] - Loss: 18.016 [-17.897, 0.119, 0.000]
Epoch 6 [186/340] - Loss: 18.065 [-17.947, 0.118, 0.000]
Epoch 6 [187/340] - Loss: 23.317 [-23.198, 0.119, 0.000]
Epoch 6 [188/340] - Loss: 19.927 [-19.808, 0.119, 0.000]
Epoch 6 [189/340] - Loss: 24.042 [-23.924, 0.118, 0.000]
Epoch 6 [190/340] - Loss: 17.294 [-17.176, 0.118, 0.000]
Epoch 6 [191/340] - Loss: 19.584 [-19.465, 0.119, 0.000]
Epoch 6 [192/340] - Loss: 20.831 [-20.713, 0.118, 0.000]
Epoch 6 [193/340] - Loss: 19.319 [-19.201, 0.119, 0.000]
Epoch 6 [194/340] - Loss: 21.386 [-21.268, 0.118, 0.000]
Epoch 6 [195/340] - Loss: 18.056 [-17.938, 0.118, 0.000]
Epoch 6 [196/340] - Loss: 19.254 [-19.136, 0.118, 0.000]
Epoch 6 [197/340] - Loss: 17.281 [-17.162, 0.119, 0.000]
Epoch 6 [198/340] - Loss: 20.566 [-20.448, 0.119, 0.000]
Epoch 6 [199/340] - Loss: 24.29

Epoch 6 [326/340] - Loss: 18.952 [-18.833, 0.120, 0.000]
Epoch 6 [327/340] - Loss: 17.877 [-17.759, 0.118, 0.000]
Epoch 6 [328/340] - Loss: 23.798 [-23.680, 0.118, 0.000]
Epoch 6 [329/340] - Loss: 18.757 [-18.639, 0.118, 0.000]
Epoch 6 [330/340] - Loss: 20.788 [-20.669, 0.118, 0.000]
Epoch 6 [331/340] - Loss: 18.956 [-18.837, 0.119, 0.000]
Epoch 6 [332/340] - Loss: 18.736 [-18.617, 0.119, 0.000]
Epoch 6 [333/340] - Loss: 16.679 [-16.561, 0.118, 0.000]
Epoch 6 [334/340] - Loss: 19.237 [-19.119, 0.118, 0.000]
Epoch 6 [335/340] - Loss: 20.809 [-20.690, 0.119, 0.000]
Epoch 6 [336/340] - Loss: 17.656 [-17.538, 0.118, 0.000]
Epoch 6 [337/340] - Loss: 18.833 [-18.715, 0.118, 0.000]
Epoch 6 [338/340] - Loss: 19.345 [-19.227, 0.118, 0.000]
Epoch 6 [339/340] - Loss: 20.662 [-20.544, 0.118, 0.000]


## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although our other tutorials demonstrate how to do this (for example, see the CIFAR tutorial).

In [21]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.max_cg_iterations(50), gpytorch.settings.use_toeplitz(False):
    preds = model(test_x)

In [22]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 7.868842601776123
